In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls drive/My\ Drive/Deep\ learning\ dataset/

In [ ]:
!unzip drive/My\ Drive/Deep\ learning\ dataset/nyu-cv-fall-2018.zip

In [ ]:
!unzip /content/train_images.zip

In [ ]:
!ls

In [ ]:
!unzip /content/test_images.zip

In [ ]:
from PIL import Image
import numpy as np
import os

train_images = []
train_labels = []

count = 0
for dirc in sorted(os.listdir('train_images')):
    if(count < 1):
        count += 1
        continue
    for img_path in os.listdir('train_images/'+str(dirc)):
        im = Image.open('train_images/'+str(dirc)+'/'+str(img_path))
        im = im.resize((100,100))
        train_images.append(np.array(im))
        train_labels.append(count-1)
    count += 1
    print('Reading '+str(count-2)+' dir')

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
train_images.shape

In [ ]:
train_labels.shape

In [ ]:
train_labels = np.reshape(train_labels, (-1,1))

In [ ]:
train_labels.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
train_labels = enc.fit_transform(train_labels).toarray()

In [ ]:
train_labels.shape

In [ ]:
train_images = train_images / 255.0

In [ ]:
from PIL import Image
import numpy as np
import os

test_images = []

count = 0
for dirc in sorted(os.listdir('test_images')):
    if(count < 1):
        count += 1
        continue
    im = Image.open('test_images/'+str(dirc))
    im = im.resize((100,100))
    test_images.append(np.array(im))
    print('Reading '+str(dirc))

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model


train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)

In [ ]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same', activation='relu')
        self.pool1 = MaxPool2D((2,2))
        self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
        self.pool2 = MaxPool2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(512, activation='relu')
        self.dropout1 = Dropout(0.4)
        self.d2 = Dense(128, activation='relu')
        self.dropout2 = Dropout(0.4)
        self.d3 = Dense(43, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout1(x)
        x = self.d2(x)
        x = self.dropout2(x)
        x = self.d3(x)
        return x

model = MyModel()

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
#         print(images.shape,labels.shape)  
        train_step(images, labels)
    
    model.save_weights('/content', save_format='tf')

    print('Epoch:',str(epoch+1),' Loss:',str(train_loss.result()),' Accuracy:',str(train_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

In [ ]:
test_images = np.array(test_images)

In [ ]:
test_images.shape

In [ ]:
test_images = test_images / 255.0

In [ ]:
model.load_weights('/content')

In [ ]:
predictions = []
count = 0
for img in test_images:
  count += 1 
  print(count)
  img = img.reshape(1,100,100,3)
  predictions.append(np.argmax(model(img),axis=1))

In [ ]:
!ls

In [ ]:
predictions = np.array(predictions)

In [ ]:
predictions.shape

In [ ]:
import pandas as pd 

df = pd.DataFrame(predictions)

In [ ]:
df.to_csv('/content/pred.csv')

In [ ]:
!ls

In [ ]:
df